# 要約 
このJupyter Notebookでは、Kaggleの「20の質問」ゲームでのLLM（大規模言語モデル）を利用したエージェントの構築とその実行に関する問題に取り組んでいます。具体的には、ゲームの質問者（guesser agent）と回答者（answerer agent）をモデル化し、ゲームの流れを管理するための関数を定義しています。

### 取り組んでいる問題
「20の質問」ゲームでは、質問者が特定のキーワードを推測するために、相手に質問をし、得られた回答から推測を行います。このノートブックでは、LLMを用いてゲーム進行中の質問や推測の生成を行い、正確かつ効率的なプレイを目指しています。また、エージェント同士のフィードバックがどのようにゲームの結果に影響を与えるかを考慮することも重要です。

### 使用されている手法とライブラリ
1. **定数の定義**: ゲームの状態やアクションを表す定数が定義されています。
2. **ライブラリのインポート**: 
   - `torch` - PyTorchライブラリを使用して、機械学習モデルを構築・管理します。
   - `transformers` - Hugging Face TransformersライブラリからT5モデルを使用して、質問の生成や推測を行います。
   - `pandas` - データ処理や管理に使います。
   - `json`、`os`、`random`、`string`など - データの管理や操作のために標準ライブラリを使用しています。

3. **エージェントの実装**:
   - **guesser_agent**: ゲームの状態やターンに応じた質問や推測を生成する関数。
   - **answerer_agent**: 質問に対して「はい」、「いいえ」、または「多分」で応答する関数。

4. **ゲームの進行管理**:
   - ゲーム状態を管理し、エージェントが交互に質問と回答を行うためのロジックが実装されています。
   - 関数には、ターンのインクリメント、ゲームの終了処理、エラー管理などが含まれています。

5. **モデルの呼び出し**: `call_llm`関数を通じて、生成モデルにプロンプトを渡し、その応答を受け取る機能を組み込んでいます。事前学習したT5モデルを使用し、GPUまたはCPUで動作します。

このように、このノートブックは「20の質問」ゲームのプレイを自動化するLLMベースのエージェントの開発に焦点を当て、必要な技術とライブラリを活用して効率的に問題を解決していくプロセスを示しています。

---


# 用語概説 
以下に、Jupyter Notebookの中で機械学習や深層学習の初心者がつまずきそうな専門用語の解説を列挙します。内容は特にマイナーなものや実務において馴染みが薄いものに焦点を当てています。

1. **LLM (Large Language Model)**:
   - 大規模言語モデルのことを指し、膨大な量のテキストデータで訓練され、言語理解や生成を行うことができるモデル。特にGPTやT5などが有名。これらのモデルは大量のパラメータを持ち、コンテクストに基づいた応答生成が可能。

2. **T5 (Text-to-Text Transfer Transformer)**:
   - Googleが開発したユニバーサルなテキストモデルであり、様々なNLPタスクを「テキストをテキストに変換する」問題として扱い、同じアーキテクチャを用いています。質問応答や翻訳、要約など多様な用途に対応しています。

3. **トークナイザー (Tokenizer)**:
   - テキストデータを数値的な形式に変換するプロセスを行うツール。モデルに入力するために単語や文をトークンという単位に分割し、対応する数値IDに変換します。言語モデルが理解可能な形に整形するための重要なステップ。

4. **エピソード (Episode)**:
   - ゲーム理論や強化学習において、環境内でエージェントが一連の行動を取る単位を指す。特定の初期状態からスタートして、最終的に終了状態に至るまでの一連のやり取りを含む。

5. **状態 (State)**:
   - ゲームや強化学習におけるエージェントや環境の現在の状況を指す。各ターンでエージェントが観測する情報を含み、その情報に基づいてアクションを選択します。

6. **アクション (Action)**:
   - エージェントが行う操作や意思決定を指す。例えば、質問をする、推測する、などの行動がこれに該当します。

7. **観察データ (Observation Data)**:
   - エージェントが環境から受け取る情報であり、現在の状態や過去の質問と回答から構成されます。エージェントはこのデータを基に次のアクションを選択します。

8. **既存データ (Existing Data)**:
   - モデルのトレーニングや評価に使用されるデータセットのこと。一般的に、モデルが学ぶための基盤となる参照データを指します。

9. **エラートラッキング (Error Tracking)**:
   - プログラムやシステムのエラーを特定し、記録するプロセス。特にテストやデバッグの段階で重要な役割を果たす。

10. **デバイス (Device)**:
    - 使用する計算資源を指し、CPUやGPU（グラフィックプロセッシングユニット）がこれに該当します。特に深層学習のモデルはGPUでの実行が一般的です。

11. **タイムアウト (Timeout)**:
    - 特定の操作が所定の時間内に完了しない場合に発生するエラー。ゲーム内の行動や応答に制限時間が設けられているシナリオを指します。

これらの解説を通して、初心者が「20の質問」ゲームに関するモデルを理解する際の助けになることを目指します。

---


# ChatGPTの助けを借りてドキュメント化されたコード

# 最初の設定
実行環境の設定を行い、状態やアクションの定数を定義し、あらかじめ定義されたリストからキーワードとその代替案をランダムに選択することで、「20の質問」ゲームを開始する準備をします。

In [ ]:
import sys  # sysモジュールをインポートします。これは、Pythonインタプリタや環境に関する情報や機能を提供します。

In [ ]:
import json  # JSONデータを扱うためのモジュールをインポートします。
import os  # オペレーティングシステムの機能を利用するためのモジュールをインポートします。
import pandas as pd  # データ操作と分析のためのpandasライブラリをインポートします。
import random  # ランダムな値を生成するためのモジュールをインポートします。
import string  # 文字列に関する便利な機能を提供するモジュールをインポートします。
import torch  # PyTorchライブラリをインポートします。

sys.path.append("/kaggle/input/llm-20-questions/llm_20_questions")  # 特定のディレクトリをPythonパスに追加します。

from keywords import KEYWORDS_JSON  # keywordsモジュールからKEYWORDS_JSONをインポートします。
from os import path  # pathをosモジュールからインポートします。
from pathlib import Path  # Pathをpathlibモジュールからインポートします。
from random import choice  # choice関数をrandomモジュールからインポートします。
from string import Template  # Templateクラスをstringモジュールからインポートします。
from transformers import T5Tokenizer, T5ForConditionalGeneration  # T5モデルに関連するトークナイザーと生成モデルをインポートします。

# モデルやデバイスに関する初期設定
llm_parent_dir = "/kaggle/input/flan-t5/pytorch/large"  # 使用するモデルの親ディレクトリを指定します。
device = None  # デバイス変数を初期化します（後でGPUまたはCPUを選択するため）。
model = None  # モデル変数を初期化します。
tokenizer = None  # トークナイザー変数を初期化します。
model_initialized = False  # モデルが初期化されたかどうかを示すフラグを初期化します。

# 定数の定義
ERROR = "ERROR"  # エラー状態を示す定数を定義します。
DONE = "DONE"  # 完了状態を示す定数を定義します。
INACTIVE = "INACTIVE"  # 非アクティブ状態を示す定数を定義します。
ACTIVE = "ACTIVE"  # アクティブ状態を示す定数を定義します。
TIMEOUT = "TIMEOUT"  # タイムアウト状態を示す定数を定義します。
GUESS = "guess"  # 推測アクションを示す定数を定義します。
ASK = "ask"  # 質問アクションを示す定数を定義します。
GUESSER = "guesser"  # 推測者を示す定数を定義します。
ANSWERER = "guesser"  # 回答者を示す定数を定義します。

# キーワードリストの取得と選択
keywords_list = json.loads(KEYWORDS_JSON)  # JSON形式のキーワードデータをパースしてリストに変換します。
keyword_cat = random.choice(keywords_list)  # ランダムにキーワードカテゴリを選択します。
category = keyword_cat["category"]  # 選択したカテゴリを取得します。
keyword_obj = random.choice(keyword_cat["words"])  # ランダムにキーワードオブジェクトを選択します。
keyword = keyword_obj["keyword"]  # 選択したキーワードを取得します。
alts = keyword_obj["alts"]  # 選択したキーワードの代替案を取得します。

# 推測者エージェント

<i>guesser_agent</i>関数は、現在のゲームの状態とターンの種類に基づいて動的なプロンプトを生成し、言語モデルを使用してゲーム内の次のアクションを生成します。具体的には、以下のような処理を行います。

1. これまでの質問と回答の履歴を構築します。
2. ターンの種類に基づいて適切なプロンプト（質問または推測）を選択します。
3. 構築したプロンプトを使って言語モデルを呼び出し、次のアクションを生成します。

In [ ]:
def guesser_agent(obs):  # 推測者エージェントの関数を定義します。obsは現在の観察データを含む引数です。
    info_prompt = """あなたは20の質問ゲームをプレイしており、質問をしてキーワード（実在または架空の人物、場所、または物）を推測しようとしています。 \nこれまでの情報は次のとおりです:\n{q_a_thread}"""  # ゲームの進行状況に関する情報を含むプロンプトを定義します。
    
    questions_prompt = """はいまたはいいえで答えられる質問を1つ聞いてください。"""  # 質問を促すプロンプトを定義します。
    
    guess_prompt = """キーワードを推測してください。正確な単語/フレーズのみを返答してください。たとえば、キーワードが[paris]だと思う場合は、[私はキーワードがparisだと思います]や[キーワードはParisですか？]とは返答せず、単に[word]と返答してください。[paris]だけを返してください。"""  # キーワードの推測を促すプロンプトを定義します。
    
    q_a_thread = ""  # 質問と回答の履歴を保存するための変数を初期化します。
    
    # 質問と回答の履歴を構築します。
    for i in range(0, len(obs.answers)):  # 現在の観察データの回答の数だけループします。
        q_a_thread = "{}Q: {} A: {}\n".format(  # Q&Aのスレッドを整形します。
            q_a_thread,
            obs.questions[i],  # 現在の質問を取得します。
            obs.answers[i]  # 現在の回答を取得します。
        )
    
    prompt = ""  # プロンプトを初期化します。
    
    # ターンの種類に応じてプロンプトを選択します。
    if obs.turnType == ASK:  # ターンの種類が質問のとき
        prompt = "{}{}".format(  # 情報プロンプトと質問プロンプトを結合します。
            info_prompt.format(q_a_thread=q_a_thread),
            questions_prompt
        )
    elif obs.turnType == GUESS:  # ターンの種類が推測のとき
        prompt = "{}{}".format(  # 情報プロンプトと推測プロンプトを結合します。
            info_prompt.format(q_a_thread=q_a_thread),
            guess_prompt
        )
    else:
        return ""  # ターンの種類が不明な場合は空の文字列を返します。
    
    return call_llm(prompt)  # 構築したプロンプトを使用してLLMを呼び出します。

# 回答者エージェント
answerer_agent関数は次のような処理を行います。

1. ターンの種類が「回答」であるかどうかをチェックします。

2. もし「回答」であれば、info_promptとanswer_question_promptを組み合わせてプロンプトを作成し、プレースホルダーを実際の値で置き換えます。

3. 生成したプロンプトを使用してcall_llm関数を呼び出し、応答を取得します。

4. ターンが「回答」でない場合は、空の文字列を返します。

5. agents辞書は、エージェントの種類をゲーム内の行動を定義する役割にマッピングします。

In [ ]:
def answerer_agent(obs):  # 回答者エージェントの関数を定義します。obsは現在の観察データを含む引数です。
    info_prompt = """あなたは20の質問ゲームで非常に正確な回答者です。質問者が推測しようとしているキーワードは[the {category} {keyword}]です。"""  # ゲームの設定に関する情報を含むプロンプトを定義します。
    
    answer_question_prompt = """次の質問には「はい」、「いいえ」、または不明な場合は「たぶん」のみで答えてください: {question}"""  # 質問への回答を促すプロンプトを定義します。
    
    if obs.turnType == "answer":  # ターンの種類が「回答」の場合
        prompt = "{}{}".format(  # 情報プロンプトと質問回答プロンプトを結合します。
            info_prompt.format(category=category, keyword=keyword),  # カテゴリとキーワードに実際の値を挿入します。
            answer_question_prompt.format(question=obs.questions[-1])  # 最新の質問を挿入します。
        )
        return call_llm(prompt)  # 構築したプロンプトを使用してLLMを呼び出し、応答を取得します。
    else: 
        return ""  # ターンが「回答」でない場合は空の文字列を返します。

# エージェントの種類を役割にマッピングする辞書を定義します。
agents = {GUESSER: guesser_agent, ANSWERER: answerer_agent}  # GUESSERは推測者エージェント、ANSWERERは回答者エージェントを指します。

# 推測者アクション

### 関数の構成要素
1. **`guessed`変数の初期化**:
   - 変数`guessed`は`False`として初期化され、キーワードが正しく推測されたかどうかを追跡します。
   ```python
   guessed = False
   ```

2. **アクションなしのチェック**:
   - `active.action`が空（アクションが取られていない）場合、`active`のステータスを`ERROR`に設定します。
   ```python
   if not active.action:
       active.status = ERROR
   ```

3. **質問ターンの処理**:
   - 観察者のターンの種類（`turnType`）が`ASK`（質問する）である場合:
     - アクション（`active.action`）は質問として扱われ、2000文字に制限されます。
     - 質問はアクティブと非アクティブの両方の観察者の質問リストに追加されます。
   ```python
   elif active.observation.turnType == ASK:
       question = active.action[:2000]
       active.observation.questions.append(question)
       inactive.observation.questions.append(question)
   ```

4. **推測ターンの処理**:
   - 観察者のターンの種類が`GUESS`（推測する）である場合:
     - アクション（`active.action`）は推測として扱われ、100文字に制限されます。
     - 推測はアクティブと非アクティブの両方の観察者の推測リストに追加されます。
   ```python
   elif active.observation.turnType == GUESS:
       guess = active.action[:100]
       active.observation.guesses.append(guess)
       inactive.observation.guesses.append(guess)
   ```

5. **キーワードが推測されたかのチェック**:
   - アクション（`active.action`）があり、キーワードが正しく推測された場合（`keyword_guessed(active.action)`）:
     - 変数`guessed`は`True`に設定されます。
     - スコアは`20 - int(step / 3)`として計算され、ここで`step`はゲーム内で取られたステップ数を表します。
     - `end_game`関数が呼び出され、アクティブと非アクティブの観察者を渡して、スコアと最終ステータス`DONE`でゲームを終了します。
   ```python
   if active.action and keyword_guessed(active.action):
       guessed = True
       score = 20 - int(step / 3)
       end_game(active, inactive, score, DONE, DONE)
   ```

6. **`guessed`変数の返却**:
   - 関数は`guessed`変数を返し、キーワードが正しく推測されたかどうかを示します。
   ```python
   return guessed
   ```

In [ ]:
def guesser_action(active, inactive, step):  # 推測者アクションの関数を定義します。activeはアクティブな観察者、inactiveは非アクティブな観察者、stepはゲームの進行ステップ数を示します。
    guessed = False  # キーワードが正しく推測されたかどうかを示す変数を初期化します。
    
    if not active.action:  # アクティブな観察者のアクションが空の場合
        active.status = ERROR  # ステータスをERRORに設定します。
        
    elif active.observation.turnType == ASK:  # ターンの種類が質問の場合
        question = active.action[:2000]  # アクション（質問）を2000文字に制限して取得します。
        active.observation.questions.append(question)  # アクティブな観察者の質問リストに追加します。
        inactive.observation.questions.append(question)  # 非アクティブな観察者の質問リストにも追加します。
        
    elif active.observation.turnType == GUESS:  # ターンの種類が推測の場合
        guess = active.action[:100]  # アクション（推測）を100文字に制限して取得します。
        active.observation.guesses.append(guess)  # アクティブな観察者の推測リストに追加します。
        inactive.observation.guesses.append(guess)  # 非アクティブな観察者の推測リストにも追加します。
        
    if active.action and keyword_guessed(active.action):  # アクションがあり、キーワードが正しく推測された場合
        guessed = True  # guessedをTrueに設定します。
        score = 20 - int(step / 3)  # スコアを計算します。ステップ数に応じてスコアが減少します。
        end_game(active, inactive, score, DONE, DONE)  # ゲームを終了する関数を呼び出します。
        
    return guessed  # 推測結果を返します。

# ゲームの終了
`end_game`関数はゲームを終了させるために次の処理を行います：
- 両方の参加者に対してキーワードとカテゴリを設定します。
- 両方の参加者に報酬を割り当てます。
- 両方の参加者のステータスを更新します。

これにより、すべての参加者がゲームの終了に関する正しい情報を持ち、彼らの状態がゲームの結論を反映するように適切に更新されることが保証されます。

In [ ]:
def end_game(active, inactive, reward, status, inactive_status):  # ゲームを終了させる関数を定義します。activeはアクティブな参加者、inactiveは非アクティブな参加者、rewardは報酬、statusはアクティブな参加者のステータス、inactive_statusは非アクティブな参加者のステータスを示します。
    active.observation.keyword = keyword  # アクティブな参加者の観察データにキーワードを設定します。
    active.observation.category = category  # アクティブな参加者の観察データにカテゴリを設定します。
    inactive.observation.keyword = keyword  # 非アクティブな参加者の観察データにキーワードを設定します。
    inactive.observation.category = category  # 非アクティブな参加者の観察データにカテゴリを設定します。
    
    active.reward = reward  # アクティブな参加者に報酬を設定します。
    inactive.reward = reward  # 非アクティブな参加者にも同じ報酬を設定します。
    
    active.status = status  # アクティブな参加者のステータスを設定します。
    inactive.status = inactive_status  # 非アクティブな参加者のステータスを設定します。

# 回答者アクション

`answerer_action`関数は、回答者が質問に対する反応を処理します。具体的には次のような処理を行います：
- アクティブな参加者にキーワードとカテゴリを設定します。
- 反応が有効かどうかをチェックし、"yes"、"no"、または"maybe"に正規化します。
- 反応が無効または空の場合は、エラーとしてゲームを終了します。
- 両方の参加者の回答リストに反応を追加します。

この関数は、回答者の反応に基づいてゲームの状態が適切に更新されることを保証し、エラーや無効な反応が発生した場合には、エラーステータスとしてゲームを終了します。

**回答の更新**

```python
active.observation.answers.append(response)  # アクティブな参加者の回答リストに反応を追加します。
inactive.observation.answers.append(response)  # 非アクティブな参加者の回答リストにも反応を追加します。
```
正規化された`response`は、両方の`active`と`inactive`の参加者の`answers`リストに追加されます。これにより、両方の参加者が回答者の反応の記録を持つことが保証されます。

In [ ]:
def answerer_action(active, inactive):  # 回答者アクションの関数を定義します。activeはアクティブな参加者、inactiveは非アクティブな参加者を示します。
    active.observation.keyword = keyword  # アクティブな参加者の観察データにキーワードを設定します。
    active.observation.category = category  # アクティブな参加者の観察データにカテゴリを設定します。
    
    response = active.action  # アクティブな参加者のアクション（応答）を取得します。
    
    if not response:  # 応答がない場合
        response = "none"  # 応答を"none"に設定します。
        end_game(active, inactive, -1, ERROR, DONE)  # エラーとしてゲームを終了します。
        
    elif "yes" in response.lower():  # 応答に"yes"が含まれる場合
        response = "yes"  # 応答を"yes"に設定します。
        
    elif "no" in response.lower():  # 応答に"no"が含まれる場合
        response = "no"  # 応答を"no"に設定します。
        
    else:  # それ以外の場合
        response = "maybe"  # 応答を"maybe"に設定します。
        end_game(active, inactive, -1, ERROR, DONE)  # エラーとしてゲームを終了します。
        
    active.observation.answers.append(response)  # アクティブな参加者の回答リストに応答を追加します。
    inactive.observation.answers.append(response)  # 非アクティブな参加者の回答リストにも応答を追加します。

# ターンのインクリメント
`increment_turn`関数は次のような処理を行います：
1. キーワードが推測されていない場合、60ステップ後にゲームを終了します。
2. ターンの種類を「質問」と「推測」間で切り替え、質問をする役割と推測をする役割を交互にします。
3. アクティブな参加者と非アクティブな参加者のステータスを更新し、次のターンに正しい参加者がアクティブになるようにします。

この関数は、ゲームがスムーズに進行し、参加者間で役割が交互に切り替わり、推測できなかった場合には適切にゲーム終了を処理することを保証します。

In [ ]:
def increment_turn(active, inactive, step, guessed):  # ターンをインクリメントする関数を定義します。activeはアクティブな参加者、inactiveは非アクティブな参加者、stepは現在のステップ数、guessedはキーワードが推測されたかを示します。
    if step == 59 and not guessed:  # ステップが59で、キーワードが推測されていない場合
        end_game(active, inactive, -1, DONE, DONE)  # エラーとしてゲームを終了します。
        
    elif active.observation.turnType == "guess":  # アクティブな参加者のターンの種類が「推測」の場合
        active.observation.turnType = "ask"  # ターンの種類を「質問」に切り替えます。
        
    elif active.observation.turnType == "ask":  # アクティブな参加者のターンの種類が「質問」の場合
        active.observation.turnType = "guess"  # ターンの種類を「推測」に切り替えます。
        active.status = INACTIVE  # アクティブな参加者のステータスを非アクティブに設定します。
        inactive.status = ACTIVE  # 非アクティブな参加者のステータスをアクティブに設定します。
        
    else:  # その他の場合
        active.status = INACTIVE  # アクティブな参加者のステータスを非アクティブに設定します。
        inactive.status = ACTIVE  # 非アクティブな参加者のステータスをアクティブに設定します。

# インタープリター

### `interpreter`関数の目的

`interpreter`関数は、2対のエージェントが関与するゲームの状態とアクションを管理します。各対はアクティブなエージェント（質問者）と非アクティブなエージェント（推測者）で構成されており、この関数はゲームの状態を更新し、各エージェントが取るべきアクションを決定し、遷移と終了条件を処理します。

### コードの説明

```python
def interpreter(state, env):
    if env.done:
        return state
```

この行は、環境（ゲーム）が終わったかどうかをチェックします。終わっている場合、関数は現在の状態を変更せずに返します。

### アクティブエージェントと非アクティブエージェントの分離

```python
    active1 = state[0] if state[0].status == ACTIVE else state[1]
    inactive1 = state[0] if state[0].status == INACTIVE else state[1]
    active2 = state[2] if state[2].status == ACTIVE else state[3]
    inactive2 = state[2] if state[2].status == INACTIVE else state[3]
```

これらの行は、各対のアクティブと非アクティブのエージェントを特定します。`state[0]`と`state[1]`は最初のエージェントのペア、`state[2]`と`state[3]`は二番目のエージェントのペアです。

### 完了ステータスの処理

```python
    if active1.status == DONE and inactive1.status == DONE:
        active1 = None
        inactive1 = None
    if active2.status == DONE or inactive2.status == DONE:
        active2 = None
        inactive2 = None
    if active1 is None and inactive1 is None and active2 is None and inactive2 is None:
        return state
```

これらの行は、もしペアの両方のエージェントが完了していれば、そのエージェントを`None`に設定します。すべてのエージェントが完了している場合、関数は現在の状態を返し、実質的に終了します。

### ステップと終了条件の処理

```python
    step = state[0].observation.step
    end_early = (active1 and active1.status) in (TIMEOUT, ERROR) or (active2 and active2.status in (TIMEOUT, ERROR))
    either_guessed = False
```

- `step`には、ゲームの現在のステップが格納されます。
- `end_early`は、いずれかのアクティブエージェントが`TIMEOUT`または`ERROR`のステータスを持つ場合、早期終了の条件をチェックします。
- `either_guessed`は、どのエージェントがキーワードを正しく推測したかをトラックするフラグです。

### アクティブ1エージェントの処理

```python
    if active1 is not None:
        guessed = False
        if active1.observation.role == GUESSER:
            guessed = guesser_action(active1, inactive1, step)
            either_guessed = guessed
        else:
            answerer_action(active1, inactive1)
        if active1.status in (TIMEOUT, ERROR):
            end_game(active1, inactive1, 0, active1.status, DONE)
        elif end_early:
            end_game(active1, inactive1, 0, DONE, DONE)
        else:
            increment_turn(active1, inactive1, step, guessed)
```

- `active1`が`None`でない場合、関数はそのエージェントの役割をチェックします。
- `active1`が推測者の場合、`guesser_action`を呼び出します。
- `active1`が回答者の場合、`answerer_action`を呼び出します。
- 次に、`TIMEOUT`や`ERROR`などの終了条件を処理し、`end_game`を呼び出します。
- `end_early`が真の場合、ゲームを終了します。
- そうでなければ、`increment_turn`を呼び出し、次のターンに進みます。

### アクティブ2エージェントの処理

```python
    if active2 is not None:
        guessed = False
        if active2.observation.role == GUESSER:
            guessed = guesser_action(active2, inactive2, step)
            either_guessed = either_guessed or guessed
        else:
            answerer_action(active2, inactive2)
        if active2.status in (TIMEOUT, ERROR):
            end_game(active2, inactive2, 0, active2.status, DONE)
        elif end_early:
            end_game(active2, inactive2, 0, DONE, DONE)
        else:
            increment_turn(active2, inactive2, step, guessed)
```

このブロックは`active1`の処理と似ていますが、`active2`と`inactive2`に対して操作を行います。

### 状態の返却

```python
    return state
```

関数は、両方のペアのエージェントのアクションと遷移を処理した後、更新された状態を返します。

### 概要

`interpreter`関数は次の処理を行います：

1. ゲームが完了しているかチェックし、完了している場合は状態を返します。
2. 各ペアのアクティブと非アクティブのエージェントを特定します。
3. 終了条件を処理し、質問と推測の役割を交互に切り替えます。
4. エージェントの現在の役割とステータスに基づいて適切な関数（`guesser_action`、`answerer_action`、`end_game`、`increment_turn`）を呼び出します。
5. ゲームの状態を更新し、返します。

この関数はゲームの流れを管理するために重要であり、各エージェントが自分の役割に基づいて正しいアクションを取ることを保証し、さまざまな終了条件を適切に処理します。

In [ ]:
def interpreter(state, env):  # ゲームの状態と環境を監視するインタープリター関数を定義します。
    if env.done:  # 環境（ゲーム）が終了している場合
        return state  # 現在の状態を返します。
    
    # アクティブおよび非アクティブエージェントを分離します。
    active1 = state[0] if state[0].status == ACTIVE else state[1]  # 最初のペアのアクティブエージェントを選択します。
    inactive1 = state[0] if state[0].status == INACTIVE else state[1]  # 最初のペアの非アクティブエージェントを選択します。
    active2 = state[2] if state[2].status == ACTIVE else state[3]  # 二番目のペアのアクティブエージェントを選択します。
    inactive2 = state[2] if state[2].status == INACTIVE else state[3]  # 二番目のペアの非アクティブエージェントを選択します。
    
    if active1.status == DONE and inactive1.status == DONE:  # 最初のペアの両方のエージェントがDONEの場合
        active1 = None  # アクティブエージェントをNoneに設定します。
        inactive1 = None  # 非アクティブエージェントをNoneに設定します。
    
    if active2.status == DONE or inactive2.status == DONE:  # 二番目のペアのいずれかのエージェントがDONEの場合
        active2 = None  # アクティブエージェントをNoneに設定します。
        inactive2 = None  # 非アクティブエージェントをNoneに設定します。
    
    if active1 is None and inactive1 is None and active2 is None and inactive2 is None:  # すべてのエージェントがNoneの場合
        return state  # 現在の状態を返します。
    
    step = state[0].observation.step  # 現在のステップを取得します。
    end_early = (active1 and active1.status) in (TIMEOUT, ERROR) or (active2 and active2.status in (TIMEOUT, ERROR))  # 早期終了の条件をチェックします。
    either_guessed = False  # いずれかのエージェントが推測したかをトラックするフラグを初期化します。

    if active1 is not None:  # アクティブな参加者が存在する場合
        guessed = False  # 推測状況を初期化します。
        if active1.observation.role == GUESSER:  # アクティブな参加者が推測者の場合
            guessed = guesser_action(active1, inactive1, step)  # 推測者のアクションを実行します。
            either_guessed = guessed  # 推測状況を更新します。
        else:  # アクティブな参加者が回答者の場合
            answerer_action(active1, inactive1)  # 回答者のアクションを実行します。
        
        if active1.status in (TIMEOUT, ERROR):  # タイムアウトまたはエラーの場合
            end_game(active1, inactive1, 0, active1.status, DONE)  # ゲームを終了します。
        elif end_early:  # 早期終了条件が成立する場合
            end_game(active1, inactive1, 0, DONE, DONE)  # ゲームを終了します。
        else:  # それ以外の場合
            increment_turn(active1, inactive1, step, guessed)  # ターンを進めます。
    
    if active2 is not None:  # アクティブな参加者が存在する場合
        guessed = False  # 推測状況を初期化します。
        if active2.observation.role == GUESSER:  # アクティブな参加者が推測者の場合
            guessed = guesser_action(active2, inactive2, step)  # 推測者のアクションを実行します。
            either_guessed = either_guessed or guessed  # 推測状況を更新します。
        else:  # アクティブな参加者が回答者の場合
            answerer_action(active2, inactive2)  # 回答者のアクションを実行します。
        
        if active2.status in (TIMEOUT, ERROR):  # タイムアウトまたはエラーの場合
            end_game(active2, inactive2, 0, active2.status, DONE)  # ゲームを終了します。
        elif end_early:  # 早期終了条件が成立する場合
            end_game(active2, inactive2, 0, DONE, DONE)  # ゲームを終了します。
        else:  # それ以外の場合
            increment_turn(active2, inactive2, step, guessed)  # ターンを進めます。
    
    return state  # 更新された状態を返します。

# レンダラー

- **`renderer`関数**:
  - ゲームの状態を反復処理します。
  - 各エージェントの役割、インタラクション、キーワード、およびスコアを出力します。
  - `GUESSER`エージェントのためのトランスクリプトを構築し、彼らの質問、回答、および推測を表示します。
  - エージェント間の可読性を向上させるために空行を印刷します。

- **追加コード**:
  - JSONファイルへのパスを構築します。
  - JSONファイルを開き、変数`specification`にロードします。

`renderer`関数は、ゲームの現在の状態を視覚化するのに役立ち、進行状況をデバッグしたり理解したりしやすくします。一方で、追加コードはJSONファイルからゲーム仕様をロードするためのものです。

In [ ]:
def renderer(state, env):  # ゲームの状態を表示する関数を定義します。
    for s in state:  # ゲームの状態を逐次処理します。
        print("role: ", s.observation.role)  # 各エージェントの役割を表示します。
        
        if s.observation.role == GUESSER:  # エージェントが推測者の場合
            transcript = ""  # トランスクリプトの初期化
            for i in range(0, len(s.observation.guesses)):  # 推測の数だけループします。
                transcript = "{}Q: {} A: {}\nG: {}\n".format(  # 質問、回答、推測をトランスクリプトに追加します。
                    transcript, s.observation.questions[i],  # 質問を追加
                    s.observation.answers[i],  # 回答を追加
                    s.observation.guesses[i]  # 推測を追加
                )
            print(transcript)  # 完成したトランスクリプトを表示します。
        
        print("keyword: ", s.observation.keyword)  # キーワードを表示します。
        print("score: ", s.reward)  # スコアを表示します。
        print("")  # 可読性向上のために空行を挿入します。
        print("")  # 可読性向上のために空行を挿入します。
        print("")  # 可読性向上のために空行を挿入します。
    
    return ""  # 空の文字列を返します。

# JSONファイルへのパスを構築します。
jsonpath = path.abspath(path.join("/kaggle/input/llm-20-questions/llm_20_questions", "llm_20_questions.json"))
with open(jsonpath) as f:  # JSONファイルを開きます。
    specification = json.load(f)  # JSONファイルの内容を読み込み、specificationに格納します。

### `html_renderer`関数

```python
def html_renderer():
    jspath = path.abspath(path.join(path.dirname(__file__), "llm_20_questions.js"))
    with open(jspath) as f:
        return f.read()
```

#### 説明

1. **JavaScriptのパスを構築**:
    ```python
    jspath = path.abspath(path.join(path.dirname(__file__), "llm_20_questions.js"))
    ```
    - `path.abspath`を使用して`llm_20_questions.js`ファイルの絶対パスを取得します。
    - `path.join`を使用して、現在のファイルのディレクトリ（`__file__`）と`llm_20_questions.js`を結合します。

2. **JavaScriptファイルを開き、内容を読み込む**:
    ```python
    with open(jspath) as f:
        return f.read()
    ```
    - `jspath`で指定されたパスにあるJavaScriptファイルを開きます。
    - `f.read()`を使用してファイルの内容全体を読み込みます。
    - 内容を文字列として返します。

この関数は、`llm_20_questions.js`という名前のJavaScriptファイルを読み込み、その内容を文字列として返します。これは、HTMLページにJavaScriptを動的に埋め込む際に便利です。

### `keyword_guessed`関数

```python
def keyword_guessed(guess: str) -> bool:
    def normalize(s: str) -> str:
        t = str.maketrans("", "", string.punctuation)
        return s.lower().replace("the", "").replace(" ", "").translate(t)
    if normalize(guess) == normalize(keyword):
        return True
    for s in alts:
        if normalize(s) == normalize(guess):
            return True
    return False
```

#### 説明

1. **`normalize`関数の定義**:
    ```python
    def normalize(s: str) -> str:
        t = str.maketrans("", "", string.punctuation)
        return s.lower().replace("the", "").replace(" ", "").translate(t)
    ```
    - `normalize`は、比較のために文字列を標準化するヘルパー関数です。
    - `str.maketrans`を使用して、文字列からすべての句読点を取り除きます。
    - `s.lower()`を使用して、文字列を小文字に変換します。
    - "the"と空白を空の文字列に置き換えることで、それらを取り除きます。
    - `translate(t)`を使用して、句読点を削除します。

2. **推測がキーワードと一致するかをチェック**:
    ```python
    if normalize(guess) == normalize(keyword):
        return True
    ```
    - `guess`と`keyword`の両方を標準化します。
    - 標準化した文字列を比較します。
    - 一致する場合は`True`を返します。

3. **推測が代替キーワードのいずれかと一致するかをチェック**:
    ```python
    for s in alts:
        if normalize(s) == normalize(guess):
            return True
    ```
    - `alts`に含まれる各代替キーワードを反復処理します。
    - 各代替キーワードを標準化し、標準化した`guess`と比較します。
    - 一致する場合は`True`を返します。

4. **一致しない場合は`False`を返す**:
    ```python
    return False
    ```
    - キーワードまたは任意の代替キーワードが推測と一致しない場合は`False`を返します。

`keyword_guessed`関数は、与えられた推測がターゲットキーワードまたはその代替キーワードのいずれかと一致するかをチェックします。標準化された文字列を使用して、句読点を削除し、小文字に変換し、スペースや"the"を取り除くことで、比較がケースインセンシティブになり、句読点や余分なスペースを無視することができます。

In [ ]:
def html_renderer():  # HTMLレンダラーの関数を定義します。
    jspath = path.abspath(path.join("/kaggle/input/llm-20-questions/llm_20_questions", "llm_20_questions.js"))  # JavaScriptファイルへの絶対パスを構築します。
    with open(jspath) as f:  # JavaScriptファイルを開きます。
        return f.read()  # ファイルの全内容を文字列として返します。

def keyword_guessed(guess: str) -> bool:  # 与えられた推測がキーワードまたはその代替キーワードと一致するかをチェックする関数を定義します。
    def normalize(s: str) -> str:  # 文字列を標準化するヘルパー関数を定義します。
        t = str.maketrans("", "", string.punctuation)  # 句読点を削除するための変換テーブルを作成します。
        return s.lower().replace("the", "").replace(" ", "").translate(t)  # 文字列を小文字にし、"the"と空白を除去し、句読点を削除します。

    if normalize(guess) == normalize(keyword):  # 推測とキーワードを標準化して比較します。
        return True  # 一致する場合はTrueを返します。
    
    for s in alts:  # 代替キーワードをループ処理します。
        if normalize(s) == normalize(guess):  # 各代替キーワードを推測と比較します。
            return True  # 一致する場合はTrueを返します。

    return False  # 一致しない場合はFalseを返します。

### `call_llm`関数

```python
def call_llm(prompt: str) -> str:
    global model_initialized
    global device
    global model
    global tokenizer
    
    if not model_initialized:
        if os.path.exists(llm_parent_dir) and len(os.listdir(llm_parent_dir)) > 0:
            dirs = os.listdir(llm_parent_dir)
            llm_dir = "{}/{}".format(llm_parent_dir, dirs[0])
            device = "cuda:0" if torch.cuda.is_available() else "cpu"
            model = T5ForConditionalGeneration.from_pretrained(llm_dir).to(device)
            tokenizer = T5Tokenizer.from_pretrained(llm_dir)
            model_initialized = True
        else:
            print("t5-flan model required to use default agents. Add any version of the large model.")
            print("https://www.kaggle.com/models/google/flan-t5/frameworks/pyTorch/variations/large.")
            raise Exception("t5-flan model required to use default agents.")
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return answer[0]
```

#### 説明

1. **グローバル変数**:
    ```python
    global model_initialized
    global device
    global model
    global tokenizer
    ```
    - グローバル変数`model_initialized`、`device`、`model`、`tokenizer`を使用することを宣言します。

2. **モデルの初期化チェック**:
    ```python
    if not model_initialized:
        if os.path.exists(llm_parent_dir) and len(os.listdir(llm_parent_dir)) > 0:
    ```
    - モデルがすでに初期化されているかを確認します。
    - 初期化されていない場合、`llm_parent_dir`ディレクトリが存在し、ファイルが含まれているかをチェックします。

3. **モデルとトークナイザーのロード**:
    ```python
    dirs = os.listdir(llm_parent_dir)
    llm_dir = "{}/{}".format(llm_parent_dir, dirs[0])
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model = T5ForConditionalGeneration.from_pretrained(llm_dir).to(device)
    tokenizer = T5Tokenizer.from_pretrained(llm_dir)
    model_initialized = True
    ```
    - `llm_parent_dir`内のディレクトリを取得します。
    - モデルディレクトリへのパスを構築します。
    - GPUが利用可能であれば`cuda:0`、そうでなければCPUを使用するようデバイスを設定します。
    - 指定されたディレクトリからT5モデルとトークナイザーをロードし、モデルを選択したデバイスに移動します。
    - `model_initialized`を`True`に設定し、次回の呼び出しで再初期化しないようにします。

4. **モデルディレクトリがない場合の処理**:
    ```python
    else:
        print("t5-flan model required to use default agents. Add any version of the large model.")
        print("https://www.kaggle.com/models/google/flan-t5/frameworks/pyTorch/variations/large.")
        raise Exception("t5-flan model required to use default agents.")
    ```
    - モデルディレクトリが存在しないか空である場合、エラーメッセージを表示し、例外を発生させます。

5. **モデルの入力の準備**:
    ```python
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    ```
    - 入力`prompt`をトークナイズし、PyTorchのテンソルに変換します。
    - テンソルを選択したデバイス（CPUまたはGPU）に移動させます。

6. **出力を生成**:
    ```python
    outputs = model.generate(**inputs)
    ```
    - トークナイズした入力からモデルを使用して出力を生成します。

7. **出力をデコード**:
    ```python
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return answer[0]
    ```
    - 生成された出力をテキストにデコードし、特別なトークンをスキップします。
    - デコードされた出力の最初の要素を最終的な答えとして返します。

### 概要

`call_llm`関数は、与えられたプロンプトに基づいて応答を生成するために事前学習されたT5モデルと対話する役割を担っています。モデルとトークナイザーが一度だけロードされて初期化されることを保証します。モデルが初期化されていない場合は、指定されたディレクトリからモデルとトークナイザーをロードし、デバイス（CPU/GPU）をセットアップし、モデルを初期化済みとしてマークします。その後、入力プロンプトをトークナイズし、モデルを使用して応答を生成し、応答をデコードして文字列として返します。モデルファイルが見つからない場合は、例外を発生させ、必要なモデルファイルを追加するための指示を提供します。

In [ ]:
def call_llm(prompt: str) -> str:  # プロンプトに基づいて応答を生成する関数を定義します。
    global model_initialized  # グローバル変数を使用することを宣言します。
    global device  # グローバル変数を使用することを宣言します。
    global model  # グローバル変数を使用することを宣言します。
    global tokenizer  # グローバル変数を使用することを宣言します。

    if not model_initialized:  # モデルが初期化されていない場合
        if os.path.exists(llm_parent_dir) and len(os.listdir(llm_parent_dir)) > 0:  # モデルディレクトリが存在し、ファイルが含まれている場合
            dirs = os.listdir(llm_parent_dir)  # ディレクトリ内のファイルをリストします。
            llm_dir = "{}/{}".format(llm_parent_dir, dirs[0])  # モデルディレクトリのパスを構築します。
            device = "cuda:0" if torch.cuda.is_available() else "cpu"  # GPUが使用可能であればGPU、それ以外はCPUを設定します。
            model = T5ForConditionalGeneration.from_pretrained(llm_dir).to(device)  # 指定されたディレクトリからT5モデルをロードし、選択したデバイスに移動します。
            tokenizer = T5Tokenizer.from_pretrained(llm_dir)  # 指定されたディレクトリからトークナイザーをロードします。
            model_initialized = True  # モデルが初期化されたことを示すフラグを設定します。
        else:  # モデルディレクトリが存在しないか空である場合
            print("t5-flan model required to use default agents. Add any version of the large model.")  # エラーメッセージを表示します。
            print("https://www.kaggle.com/models/google/flan-t5/frameworks/pyTorch/variations/large.")  # 参考リンクを表示します。
            raise Exception("t5-flan model required to use default agents.")  # 例外を発生させます。
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)  # プロンプトをトークナイズし、PyTorchのテンソルに変換してデバイスに移動します。
    outputs = model.generate(**inputs)  # モデルを使用して生成された出力を取得します。
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)  # 出力をデコードしてテキストに変換します。
    
    print(prompt)  # プロンプトを出力します（デバッグ用）。
    return answer[0]  # 最初の回答を返します。

# 実行/デバッグ

*参考文献:* 
1. [test_llm_20_questions.py](https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/llm_20_questions/test_llm_20_questions.py)
2. [ノートブックでのLLM 20 Questionsの実行/デバッグ](https://www.kaggle.com/code/rturley/run-debug-llm-20-questions-in-a-notebook)

In [ ]:
from kaggle_environments import make  # kaggle_environmentsライブラリからmake関数をインポートします。

def custom_questioner(obs):  # カスタム質問者の関数を定義します。
    if obs.turnType == "guess":  # ターンの種類が「推測」の場合
        return "banana"  # "banana"を返します。
    return "Is it a banana?"  # それ以外の場合は質問を返します。

def custom_answerer():  # カスタム回答者の関数を定義します。
    return "no"  # "no"を返します。

def bad_answerer():  # 悪い回答者の関数を定義します。
    return "maybe?"  # "maybe?"を返します。

def error_agent():  # エラーエージェントを定義します。
    raise ValueError  # ValueErrorを発生させます。

def test_llm_20_q_completes():  # LLM 20 Questionsが正常に終了するかをテストする関数を定義します。
    env = make("llm_20_questions", debug=True)  # 環境を作成します。
    game_output = env.run([guesser_agent, answerer_agent, guesser_agent, answerer_agent])  # ゲームを実行します。
    json = env.toJSON()  # 環境の状態をJSON形式に変換します。
    env.render(mode="ipython", width=400, height=400)  # 環境を表示します。
    assert json["name"] == "llm_20_questions"  # 環境名が正しいかを確認します。
    assert json["statuses"] == ["DONE", "DONE", "DONE", "DONE"]  # ステータスがすべてDONEであることを確認します。

def test_llm_20_q_errors_on_bad_answer():  # 悪い回答に対するエラーをテストする関数を定義します。
    env = make("llm_20_questions", debug=True)  # 環境を作成します。
    env.run([custom_questioner, custom_answerer, custom_questioner, bad_answerer])  # ゲームを実行します。
    json = env.toJSON()  # 環境の状態をJSON形式に変換します。
    assert json["name"] == "llm_20_questions"  # 環境名が正しいかを確認します。
    assert json["rewards"] == [1, 1, 1, None]  # 報酬が期待通りであることを確認します。
    assert json["statuses"] == ["DONE", "DONE", "DONE", "ERROR"]  # ステータスが期待通りであることを確認します。
    print(len(json["steps"]))  # ステップ数を出力します。
    assert len(json["steps"]) == 3  # ステップ数が期待通りであることを確認します。

def test_llm_20_q_errors_on_error_answer():  # 回答エラーをテストする関数を定義します。
    env = make("llm_20_questions", debug=True)  # 環境を作成します。
    env.run([custom_questioner, custom_answerer, custom_questioner, error_agent])  # ゲームを実行します。
    json = env.toJSON()  # 環境の状態をJSON形式に変換します。
    assert json["name"] == "llm_20_questions"  # 環境名が正しいかを確認します。
    assert json["rewards"] == [1, 1, 1, None]  # 報酬が期待通りであることを確認します。
    assert json["statuses"] == ["DONE", "DONE", "DONE", "ERROR"]  # ステータスが期待通りであることを確認します。
    print(len(json["steps"]))  # ステップ数を出力します。
    assert len(json["steps"]) == 3  # ステップ数が期待通りであることを確認します。

def test_llm_20_q_errors_on_error_question():  # 質問エラーをテストする関数を定義します。
    env = make("llm_20_questions", debug=True)  # 環境を作成します。
    env.run([custom_questioner, custom_answerer, error_agent, custom_answerer])  # ゲームを実行します。
    json = env.toJSON()  # 環境の状態をJSON形式に変換します。
    assert json["name"] == "llm_20_questions"  # 環境名が正しいかを確認します。
    assert json["rewards"] == [1, 1, None, 1]  # 報酬が期待通りであることを確認します。
    assert json["statuses"] == ["DONE", "DONE", "ERROR", "DONE"]  # ステータスが期待通りであることを確認します。
    print(len(json["steps"]))  # ステップ数を出力します。
    assert len(json["steps"]) == 2  # ステップ数が期待通りであることを確認します。

In [ ]:
test_llm_20_q_completes()  # LLM 20 Questionsが正常に終了するかをテストする関数を実行します。